<a href="https://colab.research.google.com/github/xaximpvp2/master/blob/main/codigo_aula17_localizacao_falta_SEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes neurais para localizar área sob falta de um sistema elétrico de potência


## Pacotes:

Importaremos aqui os seguintes pacotes:
- [numpy](https://numpy.org/) é o pacote fundamental para computação científica com Python
- [matplotlib](http://matplotlib.org) é uma biblioteca popular para plotar gráficos em Python
- [tensorflow](https://www.tensorflow.org/) é uma plataforma popular para Aprendizado de Máquina em Python

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline
# O comando '%matplotlib inline' serve para que os gráficos sejam plotados imediatamente após a célula atual

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


### Definição do Problema

Nessa atividade de programação, você irá usar uma rede neural para localizar qual área de um determinado sistema elétrico (área 1 ou área 2) encontra-se sob falta (curto-circuito monofásico). Trata-se de um problema de classificação binária.

### Conjunto de dados

Começaremos essa atividade carregando os dados

- O conjunto de dados contém 10800 amostras de treinamento de 308 sinais elétricos do sistema. Cada amostra possui um rótulo que informa se essa amostra se refere a um curto-circuito que ocorreu na área 1 do sistema ou na área 2.  

    - Cada amostra de treinamento é um instante de tempo onde 308 sinais elétricos do sistema foram medidos (fasores de tensão e de corrente, frequência de operação, fluxos de potência, etc)
    - Isso nos leva à matriz X (10800 x 308), onde cada linha é um exemplo de um instante de tempo onde os 308 sinais elétricos do sistema foram medidos.

$$X =
\left(\begin{array}{cc}
--- (x^{(1)}) --- \\
--- (x^{(2)}) --- \\
\vdots \\
--- (x^{(m=10800)}) ---
\end{array}\right)$$

- O conjunto de dados de treinamento também possui vetor `y` com dimensões 10800 x 1. Ele contém os rótulos corretos para as amostras que estão em `X`
    - `y = 0` indica que a falta ocorreu na área `1`, e `y = 1` indica que a falta ocorreu na área `2`.


In [2]:
# Carregando a matriz de características
file = open('X.csv')
X    = np.loadtxt(file, delimiter=",")
# Cada linha denota um instante de tempo onde as PMUs (Phasor Measurement Units) fizeram medições.
# Cada coluna denota um sinal elétrico diferente medido pela PMU

# Carregando vetor de classes (rótulos) correspondente
file = open('y.csv')
y    = np.loadtxt(file, delimiter=",")
y    = y.reshape((-1,1))


#### Olhando as variáveis

Vamos agora nos familiarizar com o conjunto de dados.
- Uma boa forma para começar é dar print de cada variável e ver o que ela contém

O código abaixo dá print dos elementos contidos nas variáveis `X` e `y`

In [3]:
print ('O primeiro elemento de X é: ', X[0])

O primeiro elemento de X é:  [ 6.0000e+01  1.0061e+00 -1.3667e-01  2.9200e+00 -2.8719e+00 -4.8145e-02
 -1.9173e+00  1.8966e+00  2.0643e-02  3.2000e+00 -3.1487e+00 -5.1262e-02
  1.5300e+00 -1.5158e+00 -1.4190e-02  6.0000e+01  1.0258e+00 -1.1119e-01
  3.1364e+00  2.2632e+00 -4.7942e+00 -3.1895e+00 -3.0882e-01  2.8930e+00
 -6.5488e-01  1.8953e-01  9.8029e-01  2.1653e-01  9.9143e-01 -1.7229e+00
  3.2900e+00  2.3004e+00 -5.0267e+00 -3.2957e+00 -4.2702e-01  3.1592e+00
  3.2300e-01 -4.4606e-01 -4.7245e-01  1.3255e-01 -9.8263e-01  1.4456e+00
  6.0000e+01  1.0252e+00 -1.3135e-01  2.0101e+00  2.3683e-01 -2.2469e+00
 -3.4909e-01  4.0099e-01 -5.1905e-02  2.1067e+00  1.9014e-01 -2.2968e+00
  9.3869e-02 -4.4222e-01  3.4836e-01  6.0000e+01  1.0495e+00 -1.8696e-02
  1.1696e+00 -5.9836e+00  4.8140e+00  7.4349e-02  5.9050e-01 -6.6485e-01
  1.2262e+00 -6.2910e+00  5.0649e+00 -9.9899e-02 -5.0788e-01  6.0778e-01
  6.0000e+01  9.9034e-01 -3.4802e-02  6.2969e+00 -5.0571e+00 -1.2398e+00
 -1.2613e+00  1.3401e+

In [4]:
print ('O primeiro elemento de y é: ', y[0,0], '   ---> Ou seja, trata-se da amostra de um evento que ocorreu na área 1')
print ('O elemento 1000 de y é    : ', y[1000,0], '   ---> Ou seja, trata-se da amostra de um evento que ocorreu na área 2')

O primeiro elemento de y é:  0.0    ---> Ou seja, trata-se da amostra de um evento que ocorreu na área 1
O elemento 1000 de y é    :  1.0    ---> Ou seja, trata-se da amostra de um evento que ocorreu na área 2



#### Apenas checando as dimensões das nossas variáveis

Uma outra forma de nos familiarizarmos com os nossos dados é verificar suas dimensões.

In [5]:
print ('O shape de X é: ' + str(X.shape))
print ('O shape de y é: ' + str(y.shape))

O shape de X é: (10800, 308)
O shape de y é: (10800, 1)


Abaixo Padronizamos os dados usando uma camada de normalização Tensorflow (normalização Z-Score)

In [6]:
camada_norm = tf.keras.layers.Normalization(axis=-1)
camada_norm.adapt(X)  # calcula média e variância
X_norm = camada_norm(X) # características normalizadas


### Representação do modelo

- A rede neural que você irá usar deve possuir 3 camadas do tipo `dense` com ativações do tipo relu e linear.
  - lembre-se que nossas entradas são os valores dos sinais elétricos medidos em diferentes pontos do sistema
  - Uma vez que 308 sinais elétricos foram medidos, temos um total de $308$ características de entrada
    

- A rede neural deve ter $10$ unidades na camada 1, $5$ unidades na camada 2 e $1$ unidade de saída na camada 3.

    - Lembre-se que as dimensões dos parâmetros de cada camada são conforme a seguir:
        - Se a rede possui numa camada com $s_{out}$ unidades e $s_{in}$ entradas, então
            - $W$ terá dimensão $s_{in} \times s_{out}$.
            - $b$ será um vetor com $s_{out}$ elementos
  
    - Portanto, os shapes de `W` e `b` são:
        - Camada 1: O shape de `W1` é (308, 10) e o shape de `b1` é (10,)
        - Camada 2: O shape de `W2` é (10,5) e o shape de `b2` é (5,)
        - Camada 3: O shape de `W3` é (5,1) e o shape de `b3` é (1,)
        
>**OBS:** O vetor de bias `b` poderia ser representado como uma array 1-D (n,) ou 2-D (n,1). Tensorflow usa uma representação 1-D e iremos manter essa convenção.               


### Implementação do modelo usando Tensorflow


Abaixo, usamos as funções [Sequential model](https://keras.io/guides/sequential_model/) e [Dense Layer](https://keras.io/api/layers/core_layers/dense/) do Keras para construir a rede desejada.

In [7]:
modelo = Sequential(
    [
        tf.keras.Input(shape=(308,)),    # especificando a dimensão do vetor de entrada (não é obrigatório especificar)
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(5, activation="relu"),
        tf.keras.layers.Dense(1, activation="linear")
    ], name = "meu_modelo"   # Na definição do nome do modelo, não pode haver espaços em branco
)


In [8]:
modelo.summary()

Model: "meu_modelo"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 10)                  │           3,090 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              55 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,151 (12.31 KB)

 Trainable params: 3,151 (12.31 KB)

 Non-trainable params: 0 (0.00 B)

A contagem de parâmetros acima corresponde ao número de elementos w + b do modelo, conforme detalhado abaixo.

In [9]:
num_params_camada1 = 308 * 10 + 10  # parâmetros W1 + parâmetros b1
num_params_camada2 = 10 * 5 + 5   # parâmetros W2 + parâmetros b2
num_params_camada3 = 5 * 1 + 1     # parâmetros W3 + parâmetros b3
print("Número param. camada 1 = ", num_params_camada1, ", Número param. camada 2 = ", num_params_camada2, ",  Número param. camada 3 = ", num_params_camada3 )

Número param. camada 1 =  3090 , Número param. camada 2 =  55 ,  Número param. camada 3 =  6


Vamos examinar agora com cuidado os pesos para verificar que o Tensorflow produziu as mesmas dimensões que nós havíamos calculado anteriormente.

In [10]:
[camada1, camada2, camada3] = modelo.layers

In [11]:
#### Examinando os shapes dos pesos
W1,b1 = camada1.get_weights()
W2,b2 = camada2.get_weights()
W3,b3 = camada3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

W1 shape = (308, 10), b1 shape = (10,)
W2 shape = (10, 5), b2 shape = (5,)
W3 shape = (5, 1), b3 shape = (1,)


O código a seguir define uma função de perda e roda o método do gradiente para ajustar os pesos do modelo aos dados de treinamento.

In [12]:
modelo.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

modelo.fit(
    X_norm,y,
    epochs=50
)

Epoch 1/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3.2806
Epoch 2/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8042
Epoch 3/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6953
Epoch 4/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6346
Epoch 5/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5548
Epoch 6/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4869
Epoch 7/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5725
Epoch 8/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4943
Epoch 9/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5662
Epoch 10/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8147
Epoch 11/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6910
Epoch 12/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5251
Epoch 13/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4627
Epoch 14/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4405
Epoch 15/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

Abaixo calculamos a taxa de acerto para os dados de estimação

In [13]:
Probabilidades  = modelo(X_norm)
Yhat            = (Probabilidades.numpy() >= 0.5).astype(int)
taxa_acerto_est = np.mean((Yhat==y)*100)
print(f"taxa de acerto para os dados de estimação: {taxa_acerto_est}")

taxa de acerto para os dados de estimação: 87.87962962962963


## Validando nosso modelo de localização de falta usando novos dados de falta


Abaixo calculamos a taxa de acerto para um novo conjunto de dados de eventos com 4800 amostras para verificarmos se o nosso modelo consegue acertar em qual área esse evento ocorreu, mesmo que ele não tenha tido acesso a dados desses eventos durante seu treinamento.

In [14]:
# Carregando a matriz de características
file = open('X_val.csv')
X_val    = np.loadtxt(file, delimiter=",")

file = open('y_val.csv')
y_val    = np.loadtxt(file, delimiter=",")
y_val    = y_val.reshape((-1,1))

# Aplicando a camada de normalização para esses novos dados:
X_val_norm  = camada_norm(X_val)

logit       = modelo(X_val_norm)
Previsao    = tf.nn.sigmoid(logit)
Yhat        = (Previsao.numpy() >= 0.5).astype(int)
taxa_acerto_val = np.mean((Yhat==y_val.reshape(-1,1))*100)
print(f"taxa de acerto para os dados de validação: {taxa_acerto_val}")


taxa de acerto para os dados de validação: 75.625



### Parabéns!

Você construiu e utilizou uma rede neural para classificação binária que é capaz de localizar a área sob falta em um sistema elétrico!